In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
#from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_scheduler
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                                                     
#model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=655)

In [3]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
#model_name = "sberbank-ai/sbert_base_nlu_ru"
model_name = "DeepPavlov/rubert-base-cased-sentence"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=660)
#попробуем позже

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
#load best model
#model.load_state_dict(torch.load("../../best_model3004/pytorch_model.bin", map_location=device))
#model.load_state_dict(torch.load("../../best_model2306/pytorch_model.bin", map_location=device))

<All keys matched successfully>

In [4]:
torch.cuda.empty_cache()
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [6]:
df = pd.read_csv("../data/mergedcleared1307.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [7]:
sim = pd.read_csv("../data/Similar.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [8]:
df = pd.concat([df, sim])

In [9]:
df['g_id'] = df['id'].str.slice(start=0, stop=4)

In [10]:
dscr = pd.read_csv("../data/desc4d.csv", sep=';', names=['id', 'label'])
ids = pd.Series(dscr['id'])
ids = ids.astype(str)
dscr['id'] = ids
ids = []
for i, id in enumerate(dscr['id']):
    if len(id) == 4:
        ids.append(str(id))
    else:
        ids.append('0' + str(id))
dscr['id'] = ids

In [11]:
withlabels = list(dscr['id'])
justids = list(df['g_id'].unique())
for each in justids:
    if each not in withlabels:
        print(each, len(df[df['g_id']==each]))

1062 2
1018 5
1012 5
1068 1
5022 8
5058 1
5032 1
5035 1
5052 1
5075 2
6908 1


In [12]:
df.value_counts('g_id')

g_id
4011    9821
9503    9317
8708    7594
7318    4433
9405    4421
        ... 
5032       1
5052       1
5058       1
6908       1
1068       1
Length: 660, dtype: int64

In [13]:
df = df[df['label'].notnull()]

In [14]:
from sklearn import preprocessing

In [15]:

Label_encoder = preprocessing.LabelEncoder()
Label_encoder.fit(df['g_id'])

LabelEncoder()

In [16]:
df['g_id'] = Label_encoder.fit_transform(df['g_id'])

In [17]:
df.columns = ['id', 'description', 'labels']

In [18]:
df = df.drop(['id'], axis=1, inplace=False)
df

,description,labels
0,удочки рыболовные разборные состоящие из отдел...,639
1,удочки рыболовные разборные состоящие из отдел...,639
2,удочки рыболовные штекерные карбоновые длина у...,639
3,удочки рыболовные составные состоящие из отдел...,639
4,удочки рыболовные телескопические выдвижные с...,639
...,...,...
59752,грохот sandvik,512
59753,вибрационный стол,512
59754,вибрационные грохоты для горнодобывающей промы...,512
59755,аппарат для деления руды,512


In [19]:
df.to_csv('../../cler_enc2610.csv', index=False)

In [20]:
np.save('./cl_classes2610.npy', Label_encoder.classes_)

In [21]:
df = df.sample(frac=1).reset_index(drop=True)
train = df[:int(len(df)*0.8)]
test =  df[int(len(df)*0.8):]

In [22]:
print(len(test['labels'].value_counts()))

650


In [23]:
train.to_csv('../../../train.csv', index=False)
test.to_csv('../../../test.csv', index=False)

In [24]:
torch.cuda.empty_cache()
import gc
gc.collect()

11

In [5]:
#!pip install datasets

In [25]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'train': '../../../train.csv', 'test': '../../../test.csv'})

Extracting data files: 100%|████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 397.55it/s]


Dataset csv downloaded and prepared to C:/Users/yeril/.cache/huggingface/datasets/csv/default-c58292ae098436e3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 90.89it/s]


In [26]:
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)

In [28]:
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))

In [29]:
set(dataset['train'].features.keys())


{'attention_mask', 'input_ids', 'labels', 'token_type_ids'}

In [30]:
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 

In [31]:
dataset.shape

{'train': (234186, 4), 'test': (58547, 4)}

In [40]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=4)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=4)
#если есть видеопамять около 10гб, можно поставить 8

In [41]:
len(train_dataloader)

58547

In [42]:

optimizer = AdamW(model.parameters(), lr=2e-6)

num_epochs = 15

num_training_steps = num_epochs * len(train_dataloader)


lr_scheduler = get_scheduler(
    "constant_with_warmup",
    optimizer=optimizer,
    num_warmup_steps=2000,
    num_training_steps=num_training_steps
)

In [44]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [36]:
from tqdm import tqdm

In [37]:
#next(iter(train_dataloader))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [36]:
num_epochs = 10

In [ ]:
best_f1 = 0.
show_train_loss_every_num_epoch = 0.1

for epoch in range(num_epochs):
    print(40*'-', '\nepoch', epoch+1)
    model.train()
    losses = []

    for i, batch in enumerate(tqdm(train_dataloader)):
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        losses.append(loss.item())

        #f i%int(len(train_dataloader)*show_train_loss_every_num_epoch)==int(len(train_dataloader)*show_train_loss_every_num_epoch)-1:
    print(f'train loss [{i*100/len(train_dataloader):.2f}%]: {np.array(losses).mean():.3f}')
    losses = []
    print('\nvalidating')

    f1 = load_metric('f1')
    acc = load_metric('accuracy')
    precision = load_metric('precision')
    recall = load_metric('recall')
    with torch.no_grad():
        model.eval()
                
        for batch in tqdm(test_dataloader):
        ##for batch in test_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            f1.add_batch(predictions=predictions, references=batch["labels"])    
            acc.add_batch(predictions=predictions, references=batch["labels"])    
            precision.add_batch(predictions=predictions, references=batch["labels"])    
            recall.add_batch(predictions=predictions, references=batch["labels"])    

        print('weighted summary:')
        print('Test acc:', acc.compute()['accuracy'])
        print('Test precision:', precision.compute(average = 'weighted')['precision'])
        print('Test recall:', recall.compute(average = 'weighted')['recall'])
        f1_weighted = f1.compute(average = 'weighted')['f1']
        print('Test f1:', f1_weighted, '\n')

        if f1_weighted > best_f1:
            best_f1 = f1_weighted
            model.save_pretrained("../../../best_model2306")

    model.train()

---------------------------------------- 
epoch 1


100%|██████████████████████████████████████████████████████████████████████████| 58547/58547 [4:47:57<00:00,  3.39it/s]
C:\Users\yeril\AppData\Local\Temp\ipykernel_16856\608282909.py:25: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1 = load_metric('f1')


train loss [100.00%]: 3.779

validating


100%|████████████████████████████████████████████████████████████████████████████| 14637/14637 [14:27<00:00, 16.88it/s]


weighted summary:
Test acc: 0.5943942473568244


...

100%|████████████████████████████████████████████████████████████████████████████| 14637/14637 [14:27<00:00, 16.87it/s]


weighted summary:
Test acc: 0.809349753189745


C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test precision: 0.7977102029989132
Test recall: 0.809349753189745
Test f1: 0.7912256026195871 

---------------------------------------- 
epoch 4


 96%|███████████████████████████████████████████████████████████████████████▎  | 56442/58547 [8:35:38<14:27,  2.43it/s]

In [16]:
from sklearn import preprocessing
from sklearn.metrics import classification_report
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./classes.npy', allow_pickle=True)

true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

100%|████████████████████████████████████████████████████████████████████████████| 2152/2152 [02:45<00:00, 13.01it/s]
/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        0805       0.00      0.00      0.00         9
        0808       1.00      0.38      0.55        16
        1704       0.38      1.00      0.56        80
        1905       0.00      0.00      0.00        14
        2003       0.88      1.00      0.94        15
        2007       0.00      0.00      0.00         7
        2009       0.00      0.00      0.00         8
        2105       0.00      0.00      0.00         8
        2106       1.00      0.17      0.29        30
        2202       0.00      0.00      0.00         3
        2710       0.00      0.00      0.00         8
        2918       0.00      0.00      0.00        29
        2924       0.00      0.00      0.00        27
        2933       0.19      0.76      0.31        55
        2934       0.00      0.00      0.00        36
        2941       0.00      0.00      0.00        22
        3005       1.00      1.00      1.00        14
        3206       1.00    

/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [113]:
inputs = tokenizer('покрышки', truncation = True, max_length=100, padding='max_length', return_tensors="pt")
#batch = {k: torch.tensor(v).to(device) for k, v in txt.items()}
model.to(torch.device('cpu'))
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
#Label_encoder.inverse_transform(predicted_class_id)
a = []
a.append(predicted_class_id)

In [114]:
Label_encoder.inverse_transform([a])

array(['4011'], dtype=object)

In [17]:
def predict_class(text):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        return Label_encoder.inverse_transform([predicted_class_id])


In [19]:
print(predict_class("Велосипед"), predict_class("велосипед"), predict_class("Велосипедов"))

['9503'] ['9503'] ['8714']
